In [ ]:
# Imports needed for the user machine at FLASH (as we needed to install code locally as a user)
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+".local/lib/python3.5/site-packages/")
sys.path.append("./lib")


# Modules
import sys, os, math, h5py
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pyplt
import pylab as pl
from imp import reload
import DldFlashProcessor

# fdor inline plotting
%matplotlib inline

## Handling of DldFlashProcessor Class

The DldFlashProcessor Class gets the data (predefined in the Class itself. There is a lot more data in the original file stored in the DAQ!) of the given Run from the DAQ and stores it in the Folder /home/pg2user/copiedFiles/beamtime. 

In [ ]:
# Update DldFlashProcessor class and make instance
reload(DldFlashProcessor)
processor = DldFlashProcessor.DldFlashProcessor()

In [ ]:
# Get a run from the DAQ and store it in /home/pg2user/copiedFiles/beamtime/fl1user2/
processor.readRun(186440)

In [ ]:
# Alternatively if the file has already been stored locally use the following class method:
FileName = '/home/pg2user/copiedFiles/beamtime/fl1user2/FLASH1_USER2_stream_2_run18467_file37_20170827T103301.1.h5'
processor.readHdfFile(FileName)

In [ ]:
# We can now add some appropriate binning along the following axes:

processor.addBinning('posX', 500, 1000, 2)
processor.addBinning('posY', 500, 1000, 2)
processor.addBinning('dldDetectorId', -1, 2, 1)
# useful for normalization
processor.addBinning('dldTime', 637.0, 649.0, 0.5)

# other axes are 'dldMicrobunchId' , 'bam' , 'delayStage'

# Get a multidimensional array containing the binned data
cuts = processorprojection.computeBinnedData()

In [ ]:
# Store the data in the cuts struct as individual hdf5file that can be used with ImageJ

processor.save2hdf5(cuts,'goodFilename')

### Import hdf5 file into ImageJ and convert to Tiff

After the last step the stored hdf5 file can be opened in ImageJ under File->Import
When the file has been loaded you can again in the main window use File->SaveAs to convert the imported data to whatever format that is needed

## Make a Norm Image 

In [ ]:
# We did a norm image for all measurements

processor.readRun(18440)
processor.addBinning('posX', 500, 1000, 2)
processor.addBinning('posY', 500, 1000, 2)
processor.addBinning('dldDetectorId', -1, 2, 1)
norm = processor.computeBinnedData()
norm = np.nan_to_num(norm)

norm[norm<10]=1 # 10 or smaller seems to be outside of detector
norm[:,:,0][norm[:,:,0] >= 10]/=norm[:,:,0][norm[:,:,0] >= 10].mean()
norm[:,:,1][norm[:,:,1] >= 10]/=norm[:,:,1][norm[:,:,1] >= 10].mean()
norm[norm<0.05]=0.1

## Quick Plot (Simon)

In [ ]:
reload(DldFlashProcessor)
processor = DldFlashProcessor.DldFlashProcessor()

runNumber = 18465
timeFrom = 636
timeTo = 642
timeStep = .5

delayFrom = -80
delayTo = -40
delayStep = 1.2 # with 0.5, you have lot of stripes, with 2 no stripes but everything smeared out... :-(

result, result1, diff = processor.fastpumpProbePlotCheck(runNumber, timeFrom, timeTo, timeStep, delayFrom, delayTo, delayStep)

## Plotting Images 

In [ ]:
processor.readRun(18421)
processor.addBinning('dldTime', 635, 650.25,.25)
processor.addBinning('posX', 500, 1000, 2)
processor.addBinning('posY', 500, 1000, 2)
processor.addBinning('dldDetectorId', -1, 2, 1)
cuts2 = processor.computeBinnedData()
cuts2 = np.nan_to_num(cuts2)

In [ ]:
times = np.arange(635, 650,1)
pl.figure(figsize=(17,24))
for i in range(15):
    pl.subplot2grid((5,3),(i//3,i%3))
    pl.title('dldTime of '+str('{:06.2f}'.format(times[i]))+' ns',fontsize = 16)
    tempim=cuts2[4*i:4*i+4,:,:,0].sum(axis=0).transpose()/norm[:,:,0].transpose()
    tempim+=cuts2[4*i:4*i+4,:,:,1].sum(axis=0).transpose()/norm[:,:,1].transpose()
    pl.imshow(tempim,cmap = 'inferno', clim=(0,tempim.max()*.3),aspect = 'auto')
    pl.colorbar()